In [ ]:
import pytesseract
import cv2
from textblob import TextBlob
import numpy as np
from pytesseract import Output
import re
from collections import defaultdict
from PIL import Image
import imagehash
import pickle

# Tesseract

In [ ]:
# https://nanonets.com/blog/ocr-with-tesseract/
# get grayscale image
def get_grayscale(image):
    kernel = np.ones((5,5), np.uint8)
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, kernel)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5), np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5), np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5), np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 

In [ ]:
kernel = np.ones((5,5), np.uint8)
noise = cv2.medianBlur(img, 5)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY, kernel)
erosion = cv2.erode(gray, kernel, iterations = 1)
dilation = cv2.dilate(gray, kernel, iterations = 1)
opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
closing = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)
tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, kernel)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

In [ ]:
for idx, morph in enumerate([noise, gray, erosion, dilation, opening, closing, gradient, tophat, blackhat]): #[gray, thresh, opening, canny, deskewed]:
    print(idx)
    d = pytesseract.image_to_data(morph, output_type=Output.DICT)
    words = [w for w in d['text']]# if len(w) > 2]
    print(' '.join(words))
    s = ''
    for t in words:
        try:
            t = re.match('(\w+)', t).group(0)
            correction = TextBlob(t.lower()).correct().raw
            if correction in word_list:
                s += correction + ' '
        except:
            t = ''
    print(s, '\n')

# Google Vision API

In [ ]:
from google.cloud import vision
from google.oauth2 import service_account
from dotenv import load_dotenv
import os


project_folder = os.path.expanduser('../../') # the folder of your project
load_dotenv(os.path.join(project_folder, '.env'))

GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
credentials = service_account.Credentials.from_service_account_file(project_folder + GOOGLE_APPLICATION_CREDENTIALS)

client_options = {'api_endpoint': 'eu-vision.googleapis.com'}

client = vision.ImageAnnotatorClient(client_options=client_options, credentials=credentials)

In [ ]:
import io
path = '/Users/faustina/Documents/Instagram/minaa_b/2018-04-22_12-26-34_UTC.jpg'
with io.open(path, 'rb') as image_file:
        content = image_file.read()

image = vision.types.Image(content=content)

response = client.text_detection(image=image)
texts = response.text_annotations
print('Texts:')

for text in texts:
    print('\n"{}"'.format(text.description))

    vertices = (['({},{})'.format(vertex.x, vertex.y)
                for vertex in text.bounding_poly.vertices])

    print('bounds: {}'.format(','.join(vertices)))